In [1]:
import os
import sys
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D
from tensorflow.keras.layers import MaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold

In [2]:
# ID    UTTERANCE    CORE RELATIONS
trainFile = pd.read_csv('dataset/hw1_train.csv')
# ID    UTTERANCE
testFile = pd.read_csv('dataset/hw1_test.csv')

In [3]:
# Data preparation

# input features
X = list(trainFile['UTTERANCE'])

unique_relations = []
for relation_str in trainFile['CORE RELATIONS']:
    relations = relation_str.split(' ')
    for relation in relations:
        if relation not in unique_relations:
            unique_relations.append(relation)

unique_nodes = []
for path in unique_relations:
    nodes = path.split('.')
    for node in nodes:
        if node not in unique_nodes:
            unique_nodes.append(node)

# add label columns to dataframe
for relation in unique_relations:
    trainFile[relation] = 0

# fill out label columns
for idx, relation_str in enumerate(trainFile['CORE RELATIONS']):
    relations = relation_str.split(' ')
    for relation in relations:
        trainFile.loc[idx,relation] = 1

#separate label columns
labels = trainFile[unique_relations]

# target values
y = labels.values

In [4]:
# 5000 words because the training set has almost 2500 unique words
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_matrix(X)

In [5]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [6]:
def create_model(hidden_layers=[512, 512], activations=['relu', 'relu'], dropouts=[0.3, 0.3], 
                 output_activation='sigmoid', loss='binary_crossentropy'):
    
    model = Sequential()
    model.add(Dense(hidden_layers[0], input_shape=(5000,), activation=activations[0]))
    model.add(Dropout(dropouts[0]))
    for i in range(len(hidden_layers)-1):
        model.add(Dense(hidden_layers[i], activation=activations[i]))
        model.add(Dropout(dropouts[i]))
    model.add(Dense(46, activation=output_activation))
    model.compile(loss=loss, optimizer='adam', metrics=['accuracy', f1_score])
    # categorical_crossentropy, binary_crossentropy f1_loss
    return model

In [7]:
def cv_model_fit(X, y):
    kfold = KFold(n_splits=10, shuffle=True)
    scores=[]
    for train_idx, test_idx in kfold.split(X):
        model = create_model()
        model.fit(X[train_idx], y[train_idx], batch_size=32, epochs=25, verbose=0)
        score = model.evaluate(X[test_idx], y[test_idx], verbose=0)
        scores.append(score[2]*100) # f_score
        print('fold ', len(scores), '  score: ', scores[-1])
        del model
    return scores

In [8]:
cvscores = cv_model_fit(X, y)
print([np.mean(cvscores).round(2), np.std(cvscores).round(2)])

fold  1   score:  85.6295645236969
fold  2   score:  80.89064955711365
fold  3   score:  85.08695363998413
fold  4   score:  84.84514355659485
fold  5   score:  84.48495864868164
fold  6   score:  85.81226468086243
fold  7   score:  83.19141268730164
fold  8   score:  84.53482985496521
fold  9   score:  79.93490099906921
fold  10   score:  84.38608646392822


NameError: name 'scores' is not defined

In [12]:
# build last model from full data
model = create_model()
model.fit(X, y, batch_size=32, epochs=25, verbose=1)

Train on 3338 samples
Epoch 1/25
3338/3338 [==============================] - 1s 221us/sample - loss: 0.1646 - accuracy: 0.9654 - f1_score: 0.0024
Epoch 2/25
3338/3338 [==============================] - 0s 82us/sample - loss: 0.0621 - accuracy: 0.9807 - f1_score: 0.3492
Epoch 3/25
3338/3338 [==============================] - 0s 82us/sample - loss: 0.0349 - accuracy: 0.9892 - f1_score: 0.7361
Epoch 4/25
3338/3338 [==============================] - 0s 80us/sample - loss: 0.0230 - accuracy: 0.9929 - f1_score: 0.8398
Epoch 5/25
3338/3338 [==============================] - 0s 84us/sample - loss: 0.0159 - accuracy: 0.9952 - f1_score: 0.8971
Epoch 6/25
3338/3338 [==============================] - 0s 81us/sample - loss: 0.0121 - accuracy: 0.9963 - f1_score: 0.9218
Epoch 7/25
3338/3338 [==============================] - 0s 81us/sample - loss: 0.0091 - accuracy: 0.9972 - f1_score: 0.9397
Epoch 8/25
3338/3338 [==============================] - 0s 82us/sample - loss: 0.0071 - accuracy: 0.9979 - f1

In [13]:
X_t = list(testFile['UTTERANCE'])
X_pred = tokenizer.texts_to_matrix(X_t)

In [14]:
y_pred = model.predict(X_pred)

In [15]:
predictions = []
for y in y_pred:
    temp = ''
    for i,v in enumerate(y):
        if v > 0.5:
            temp += ' ' + unique_relations[i]
    if len(temp) < 1: temp = unique_relations[np.argmax(y)]
    # if len(temp)>= 2: print(temp)
    predictions.append(temp.strip())
# predictions

In [16]:
submissionFile = pd.DataFrame({
    'ID': [i for i in range(len(predictions))],
    'CORE RELATIONS': predictions})
path_to_save = os.path.abspath(os.getcwd()) + '/predictions/'
submissionFile.to_csv('predictions/prediction12.csv', index = None)
submissionFile.head()

,ID,CORE RELATIONS
0,0,movie.starring.actor
1,1,movie.starring.actor
2,2,movie.starring.actor
3,3,movie.starring.actor
4,4,movie.starring.actor
